In [1]:
#importing libraries 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from sklearn.linear_model import LinearRegression


In [2]:
#read in data
data = pd.read_csv('data/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
#data cleaning
# lets start cleaning! get your mop & broom! 

#make a copy of the dataset, we never want to overwrite our OG data 
df = data.copy()
# pull out grade code 
df['grade_num'] = data['grade'].map(lambda x:x.split(' ', 1)[0])
#bed bath ratio 
df['bed_bath_ratio'] = df['bedrooms']/df['bathrooms']
#dropping unnecessary columns: give reasons for why 
df = df.drop(columns = ['sqft_basement', 'yr_renovated', 'lat', 'long', 'view', 'condition','grade', 'date'])


#let's dummy variable for city --> 1 for in city of seatlle, 0 for outside of city limits 
#creating a list of zipcodes thar are IN the city limits of seatlle
zip_city_list = [98101, 98177, 98133, 98155, 98125, 98117, 98103, 98107, 98105, 98195, 98199, 98119, 98109, 98102, 98112, 98121, 98122, 98104, 98134, 98144, 98136, 98126, 98106, 98108, 98118, 98146, 98178]
#if it is within our city limits give it a 1,without give it 0 
df['zip_city'] = np.where(df['zipcode'].isin(zip_city_list),1,0)



#let's dummy waterfront 
# assumotion: ATM it's that it's unknown 
    #NaN meaning?--> express this as a % of missing --> report this as a data limitation; llimitations might be that some of the nans might have unreported waterfronts, based off this analyssis we will do --> we could do a ttest with a random sample of Nans and see how mnay of them do have a water front --> we have this % certainty that these nans are nos
df['waterfront'] = df['waterfront'].fillna("Unknown")
#getting dummy variables for waterfront 0= no water, 1 = yes
df1 = pd.get_dummies(df['waterfront'])
#adding dummies into a main dataframe 
main_df = pd.concat([df, df1], axis=1)
#cleaning up the columns so that we have a cleam copy
main_df['Waterfront'] = main_df['YES'].copy()
#dropping unneeded columns 
main_df = main_df.drop(columns = ['NO', 'Unknown', 'YES', 'waterfront', 'zipcode', 'yr_built'])
#store the mean of price because that is the best predictor 
mean_price = main_df['price'].mean()
main_df['mean_price'] = main_df['price'].mean()

main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              21597 non-null  int64  
 1   price           21597 non-null  float64
 2   bedrooms        21597 non-null  int64  
 3   bathrooms       21597 non-null  float64
 4   sqft_living     21597 non-null  int64  
 5   sqft_lot        21597 non-null  int64  
 6   floors          21597 non-null  float64
 7   sqft_above      21597 non-null  int64  
 8   sqft_living15   21597 non-null  int64  
 9   sqft_lot15      21597 non-null  int64  
 10  grade_num       21597 non-null  object 
 11  bed_bath_ratio  21597 non-null  float64
 12  zip_city        21597 non-null  int64  
 13  Waterfront      21597 non-null  uint8  
 14  mean_price      21597 non-null  float64
dtypes: float64(5), int64(8), object(1), uint8(1)
memory usage: 2.3+ MB


In [4]:
#remove outliers

#make a copy of the clean dataframe 
no_out = main_df.copy()

#drop columns that we cannot use 
no_out = no_out.drop(columns= ['id', 'zip_city', 'Waterfront'], axis=1)

#change data type so that we can math 
no_out = no_out.astype('float')

#pull out the columns 
columns = no_out.columns

#for each column in the dataframe, get the mean and standard deviation 
#then get the z-score for within 3 standard devaitions
for col in columns:
        
        mean = no_out[col].mean()
        sd = no_out[col].std()
        
        no_out = no_out[(no_out[col] <= mean+(3*sd))]
        
pd.set_option('display.float_format', lambda x: '%.5f' % x)
no_out.info()


#part of this loop is courtesy of Stephen Allwright 

#https://stephenallwright.com/remove-outliers-pandas/

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19479 entries, 0 to 21596
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           19479 non-null  float64
 1   bedrooms        19479 non-null  float64
 2   bathrooms       19479 non-null  float64
 3   sqft_living     19479 non-null  float64
 4   sqft_lot        19479 non-null  float64
 5   floors          19479 non-null  float64
 6   sqft_above      19479 non-null  float64
 7   sqft_living15   19479 non-null  float64
 8   sqft_lot15      19479 non-null  float64
 9   grade_num       19479 non-null  float64
 10  bed_bath_ratio  19479 non-null  float64
 11  mean_price      19479 non-null  float64
dtypes: float64(12)
memory usage: 1.9 MB


In [5]:
#add the no_out data back into the full clean dataframe 

not_out = main_df.copy()

#drop the columns that we have cleaned for outliers 
not_out = not_out.drop(columns= ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'grade_num', 'bed_bath_ratio','mean_price'], axis=1)

#add the outliers & no-outlier data frames together so that we have all our columns again 

#we could have done a merge here (left right or inner)
clean_df = pd.concat([no_out, not_out], axis=1)

#drop the null values (for the columns that we did not do the outlier math on )
clean_df = clean_df.dropna()

#check to make sure that we have all our columns with the same amount of data
clean_df.info()

#store our cleaned data as a csv file for future use
clean_df.to_csv('clean_df.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19479 entries, 0 to 21596
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           19479 non-null  float64
 1   bedrooms        19479 non-null  float64
 2   bathrooms       19479 non-null  float64
 3   sqft_living     19479 non-null  float64
 4   sqft_lot        19479 non-null  float64
 5   floors          19479 non-null  float64
 6   sqft_above      19479 non-null  float64
 7   sqft_living15   19479 non-null  float64
 8   sqft_lot15      19479 non-null  float64
 9   grade_num       19479 non-null  float64
 10  bed_bath_ratio  19479 non-null  float64
 11  mean_price      19479 non-null  float64
 12  id              19479 non-null  int64  
 13  zip_city        19479 non-null  int64  
 14  Waterfront      19479 non-null  uint8  
dtypes: float64(12), int64(2), uint8(1)
memory usage: 2.2 MB


In [6]:
#scale the data 
clean_df = pd.read_csv('clean_df.csv')

clean_df = clean_df.drop(columns = ['Unnamed: 0'])


# Instantiate a new scaler
scaler = StandardScaler()
#.invert_transform

# scaling for STD 
scaler.fit(clean_df)
#scaler.invert_transform(scaled_df)


#Apply the scale to all the data 
scaled_df = scaler.transform(clean_df)


scaled_df = pd.DataFrame(scaled_df,
                              columns=clean_df.columns,
                              index=clean_df.index)
scaled_df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_living15,sqft_lot15,grade_num,bed_bath_ratio,mean_price,id,zip_city,Waterfront
0,-1.11572,-0.35178,-1.55982,-1.06182,-0.38158,-0.87574,-0.73923,-0.97564,-0.42036,-0.53006,2.24346,0.00000,0.85470,1.39436,-0.05696
1,0.20606,-0.35178,0.28724,0.85057,-0.22479,0.97252,0.73756,-0.37210,-0.14983,-0.53006,-0.70765,0.00000,0.60609,1.39436,-0.05696
2,-1.29093,-1.52070,-1.55982,-1.62591,0.04682,-0.87574,-1.35083,1.40403,-0.09230,-1.53621,0.47280,0.00000,0.33406,-0.71718,-0.05696
3,0.48204,0.81715,1.39548,0.01132,-0.44559,-0.87574,-0.93315,-0.94115,-0.50876,-0.53006,-0.70765,0.00000,-0.75892,1.39436,-0.05696
4,0.08897,-0.35178,-0.08217,-0.37391,-0.14227,-0.87574,0.00662,-0.18242,-0.16833,0.47608,-0.41254,0.00000,-0.94412,-0.71718,-0.05696


# Regressions

In [7]:
import numpy as np
from sklearn.dummy import DummyRegressor

dummymodel = DummyRegressor(strategy = 'mean')

X = scaled_df.drop(labels = ['grade_num', 'price'], axis=1)
y = scaled_df.price

print(dummymodel.fit(X,y))

print(dummymodel.score(X,y))

y_pred = dummymodel.predict(X)

print(y_pred)

print(y.mean)

print(y - y_pred)

DummyRegressor()
0.0
[-2.04273285e-17 -2.04273285e-17 -2.04273285e-17 ... -2.04273285e-17
 -2.04273285e-17 -2.04273285e-17]
<bound method Series.mean of 0       -1.11572
1        0.20606
2       -1.29093
3        0.48204
4        0.08897
          ...   
19474   -0.53825
19475   -0.37099
19476   -0.36221
19477   -0.37099
19478   -0.68461
Name: price, Length: 19479, dtype: float64>
0       -1.11572
1        0.20606
2       -1.29093
3        0.48204
4        0.08897
          ...   
19474   -0.53825
19475   -0.37099
19476   -0.36221
19477   -0.37099
19478   -0.68461
Name: price, Length: 19479, dtype: float64


In [8]:
X6a = scaled_df[['grade_num','sqft_living']]
X6a = sm.add_constant(X)
y6a = scaled_df['price']

model6 = ols('price ~ grade_num + sqft_living', data=scaled_df)
model6_results = model6.fit()
print(model6_results.summary())

#the Durbin-Watson score is 1.955, meaning that autocorrelation is likley not a cause for concern here. therefore the independence assumptions has been met. 
params = model6_results.params
#model6_results.predict(exog= "grade_num + sqft_living"))
    
     #y_hat
#y-true(price) - y_hat --> sqaure that --> root that 
#RMSE is off by so many errors 

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.441
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     7670.
Date:                Thu, 15 Sep 2022   Prob (F-statistic):               0.00
Time:                        08:31:11   Log-Likelihood:                -21982.
No. Observations:               19479   AIC:                         4.397e+04
Df Residuals:                   19476   BIC:                         4.399e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    -3.89e-18      0.005  -7.26e-16      

In [9]:
#model 24 
# same as model 23 but dropping floors 
# mulit: price & grade_num + swft_liv + water + zip + sqftliv15 + sqftabove 
X = scaled_df[['grade_num', 'sqft_living', 'Waterfront', 'sqft_living15', 'sqft_above', 'zip_city']]
X = sm.add_constant(X)
y = scaled_df['price']

model24 = ols('price ~  grade_num + sqft_living + Waterfront + sqft_living15 + sqft_above + zip_city', data=scaled_df)
model24_results = model24.fit()
print(model24_results.summary())

#best R-sqaured so far!!
#the Durbin-Watson score is 1.950, meaning that autocorrelation is likley not a cause for concern here. therefore the independence assumptions has been met. 

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.520
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     3510.
Date:                Thu, 15 Sep 2022   Prob (F-statistic):               0.00
Time:                        08:32:08   Log-Likelihood:                -20499.
No. Observations:               19479   AIC:                         4.101e+04
Df Residuals:                   19472   BIC:                         4.107e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.863e-17      0.005  -3.75e-15